In [12]:
import os
import cv2 as cv
import numpy as np

people = ["LHL", "ZXB"]
dir = "TrainFaces"

features = []
labels = []
haar_cascade = cv.CascadeClassifier("FrontFaceHaarcascade.xml")


def create_train():
    for person in people:
        path = os.path.join(dir.encode("utf-8").decode("utf-8"), person)
        label = people.index(person)
        if not os.path.exists(path):
            print(f"Path does not exist: {path}")
            continue
        for img in os.listdir(path):
            img_path = os.path.join(path, img)
            img_array = cv.imdecode(
                np.fromfile(img_path, dtype=np.uint8), cv.IMREAD_COLOR
            )  # 解决imread不能读取中文路径的问题
            if img_array is None:
                print(f"Failed to read image: {img_path}")
                continue
            gray = cv.cvtColor(img_array, cv.COLOR_BGR2GRAY)
            faces_rect = haar_cascade.detectMultiScale(
                gray, scaleFactor=1.1, minNeighbors=4
            )
            for x, y, w, h in faces_rect:
                faces_roi = gray[y : y + h, x : x + w]
                features.append(faces_roi)
                labels.append(label)


create_train()
print("Training done ----------------")
face_recognizer = cv.face.LBPHFaceRecognizer_create()
features = np.array(features, dtype="object")
labels = np.array(labels)
# Train the Recognizer on the features list and the labels list
face_recognizer.train(features, labels)
np.save("features.npy", features)
np.save("labels.npy", labels)
face_recognizer.save("face_trained.yml")  # 能够保存训练好的模型，在其他地方直接调用

Training done ----------------


In [20]:
import numpy as np
import cv2 as cv

harr_cascade = cv.CascadeClassifier("FrontFaceHaarcascade.xml")

features = np.load("features.npy", allow_pickle=True)
labels = np.load("labels.npy")
face_recognizer = cv.face.LBPHFaceRecognizer_create()
face_recognizer.read("face_trained.yml")

people = ["LHL", "ZXB"]
path = "TrainFaces\\LHL\\2.jpg"
path1 = "Photos/7.png"

img = cv.imdecode(np.fromfile(path1, dtype=np.uint8), cv.IMREAD_COLOR)

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
cv.imshow("Person", gray)

# Detect the face in the image
faces_rect = harr_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
for x, y, w, h in faces_rect:
    face = gray[y : y + h, x : x + w]
    label, confidence = face_recognizer.predict(face)
    print(f"Label = {people[label]} with a confidence of {confidence}")
    cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), thickness=2)
    cv.putText(
        img,
        str(people[label]),
        (x - 20, y - 20),
        cv.FONT_HERSHEY_COMPLEX,
        1.0,
        (0, 255, 0),
        thickness=2,
    )
    cv.imshow("Detected", img)
    cv.waitKey(0)
cv.destroyAllWindows()

Label = LHL with a confidence of 130.72697039890056
